In [25]:
import pandas as pd
import numpy as np

In [45]:
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
anime_data = pd.read_csv("./../data/fixed/anime_full.csv")

In [193]:
characters = pd.read_csv('./../data/fixed/character_full.csv')

In [194]:
characters.head()

,ID,CharName,Gender,HairColor,Tags,Description,url,MAL_ID
0,0,L,Male,Black,"Analytical,Barefoot,Detectives,EyeBags,SweetTooth","Secretive, meticulous and cunning, L's desire ...",https://www.anime-planet.com/characters/l-deat...,1535.0
1,98,Kakashi HATAKE,Male,Grey,"Adults,Anti-GravityHair,Beastmasters,ExoticEye...",Kakashi is a jounin ninja from Konoha who is k...,https://www.anime-planet.com/characters/kakash...,34566.0
2,348,Light YAGAMI,Male,Brown,"Analytical,BigEgo,Charismatic,GodComplex,Manip...","Armed with the power of Ryuk's Death Note, Lig...",https://www.anime-planet.com/characters/light-...,1535.0
3,279,Naruto UZUMAKI,Male,Blonde,"Beastmasters,FaceMarkings,Headbands,Hot-Headed...",Naruto is a young ninja with a single dream: t...,https://www.anime-planet.com/characters/naruto...,35072.0
4,103,Edward ELRIC,Male,Blonde,"Alchemists,ArmBlades,ArtificialLimbs,Braids,Gl...",Known as a prodigy and one of the strongest al...,https://www.anime-planet.com/characters/edward...,121.0


In [204]:
characters[characters['ID']==1516]

,ID,CharName,Gender,HairColor,Tags,Description,url,MAL_ID
326,1516,Hiruzen SARUTOBI,Male,White,"Beastmasters,Elderly,FacialHair,Hats,Ninja,Pip...",Hiruzen Sarutobi is the Third Hokage of Konoha...,https://www.anime-planet.com/characters/hiruze...,20.0


In [14]:
anime_data = anime_data.drop('synopsis', axis = 1)

# 장르'만' 가지고 유사도 측정

In [11]:
genre_set = set()

In [12]:
for idx in anime_data.index:
    genres = anime_data.loc[idx, 'Genres'].split(',')
    for genre in genres:
        genre_set.add(genre)

In [29]:
genre_list = list(genre_set)

In [26]:
score = np.zeros((anime_data['MAL_ID'].nunique(), len(genre_set)))

In [38]:
for idx in anime_data.index:
    temp = anime_data.loc[idx, 'Genres'].split(',')
    for i in range(len(genre_list)):
        if genre_list[i] in temp:
            score[idx,i] = 1.0

In [46]:
cosine_similar = cosine_similarity(score, score)
for i in range(len(cosine_similar)):
    cosine_similar[i,i] = 1.0

In [47]:
cosine_similar_df = pd.DataFrame(cosine_similar)

In [256]:
def ani_rec_genre(df, cosine_similar_df, ani_name, top_n = 20):
    ani_idx = anime_data[anime_data['Name'] == ani_name].index.values
    sorted_df = cosine_similar_df[ani_idx]
    sorted_df = sorted_df.sort_values(list(ani_idx), ascending = False)
    top_n_idx = sorted_df.index[:top_n+1]
    top_n_idx_without_input = list(set(top_n_idx) - set(ani_idx))
    print(anime_data.loc[top_n_idx_without_input][['Name', 'MAL_ID']])
    #print(anime_data.loc[top_n_idx_without_input]['MAL_ID'].values)

In [189]:
ani_rec_genre(anime_data, cosine_similar_df, 'Naruto')

array(['Naruto: Shippuuden Movie 6 - Road to Ninja',
       'Boruto: Naruto the Movie - Naruto ga Hokage ni Natta Hi',
       'Dragon Ball Z: Summer Vacation Special', 'Rekka no Honoo',
       'Dragon Ball Z: Atsumare! Gokuu World',
       'Boruto: Jump Festa 2016 Special', 'Dragon Ball Kai (2014)',
       'Naruto Soyokazeden Movie: Naruto to Mashin to Mitsu no Onegai Dattebayo!!',
       'Naruto: Takigakure no Shitou - Ore ga Eiyuu Dattebayo!',
       'Naruto: Honoo no Chuunin Shiken! Naruto vs. Konohamaru!!',
       'Naruto: Shippuuden', 'Dragon Ball Kai',
       'Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsugu Mono',
       'Dragon Ball Z', 'Dragon Ball Z Movie 15: Fukkatsu no "F"',
       'Boruto: Naruto the Movie',
       'Naruto: Shippuuden Movie 4 - The Lost Tower',
       'Dragon Ball GT: Gokuu Gaiden! Yuuki no Akashi wa Suushinchuu',
       'Dragon Ball Super',
       'Dragon Ball Z Movie 11: Super Senshi Gekiha!! Katsu no wa Ore da'],
      dtype=object)

# 장르, Type, Source, Year 합산
year은 nan이 많아서 생략

In [231]:
anime_data.loc[10]

MAL_ID                                                   20
Name                                                 Naruto
Genres    Action,Adventure,Comedy,SuperPower,MartialArts...
Type                                                     TV
Source                                                Manga
Year                                                 2000.0
Name: 10, dtype: object

In [155]:
type_set = set()
for idx in anime_data.index:
    type_set.add(anime_data.loc[idx, 'Type'])
type_list = list(type_set)

source_set = set()
for idx in anime_data.index:
    source_set.add(anime_data.loc[idx, 'Source'])
source_list = list(source_set)

In [156]:
score_type = np.zeros((anime_data['MAL_ID'].nunique(), len(type_set)))
score_source = np.zeros((anime_data['MAL_ID'].nunique(), len(source_set)))

In [166]:
for idx in anime_data.index:
    temp = anime_data.at[idx,'Type']
    for i in range(len(type_list)):
        if type_list[i] == temp:
            score_type[idx,i] = 1.0

In [169]:
for idx in anime_data.index:
    temp = anime_data.at[idx,'Source']
    for i in range(len(source_list)):
        if source_list[i] == temp:
            score_source[idx,i] = 1.0

In [172]:
score_source.shape

(12181, 15)

In [173]:
source_list

['Manga',
 'Music',
 'Book',
 'Card game',
 '4-koma manga',
 'Other',
 'Novel',
 'Visual novel',
 'Picture book',
 'Radio',
 'Game',
 'Light novel',
 'Original',
 'Web manga',
 'Digital manga']

In [174]:
score.shape

(12181, 42)

In [175]:
score_type.shape

(12181, 6)

In [176]:
score_source.shape

(12181, 15)

In [177]:
score_full = np.hstack([score, score_type, score_source])

In [178]:
score_full.shape

(12181, 63)

In [209]:
cosine_similar_full = cosine_similarity(score_full, score_full)
for i in range(len(cosine_similar_full)):
    cosine_similar_full[i,i] = 1.0

In [210]:
cosine_similar_full_df = pd.DataFrame(cosine_similar_full)

In [257]:
ani_rec_genre(anime_data, cosine_similar_full_df, 'Naruto')

                                                    Name  MAL_ID
4352                                          Medaka Box   11761
131                                          Shaman King     154
8010                     Boruto: Naruto Next Generations   34566
3212                Dragon Ball Z: Atsumare! Gokuu World    6714
203                                       Rekka no Honoo     238
5518                              Dragon Ball Kai (2014)   22777
8268                     Boruto: Jump Festa 2016 Special   35072
1274                              Katekyo Hitman Reborn!    1604
1366                                  Naruto: Shippuuden    1735
1178                                          D.Gray-man    1482
3035                                     Dragon Ball Kai    6033
233                                               Bleach     269
684                                        Dragon Ball Z     813
428                                      Ueki no Housoku     479
1329                     

In [258]:
ani_rec_genre(anime_data, cosine_similar_df, 'Naruto')

                                                   Name  MAL_ID
4611         Naruto: Shippuuden Movie 6 - Road to Ninja   13667
6980  Boruto: Naruto the Movie - Naruto ga Hokage ni...   32365
5509             Dragon Ball Z: Summer Vacation Special   22695
203                                      Rekka no Honoo     238
3212               Dragon Ball Z: Atsumare! Gokuu World    6714
8268                    Boruto: Jump Festa 2016 Special   35072
5518                             Dragon Ball Kai (2014)   22777
4176  Naruto Soyokazeden Movie: Naruto to Mashin to ...   10659
533   Naruto: Takigakure no Shitou - Ore ga Eiyuu Da...     594
4181  Naruto: Honoo no Chuunin Shiken! Naruto vs. Ko...   10686
1366                                 Naruto: Shippuuden    1735
3035                                    Dragon Ball Kai    6033
3114  Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsu...    6325
684                                       Dragon Ball Z     813
5806            Dragon Ball Z Movie 15: 